# Explore GRS images

## Load python libraries

In [1]:

import glob
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray  # activate the rio accessor

import hvplot.xarray
import pyproj as ppj

#import cartopy
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import locale
#locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('TkAgg')

import grstbx
from grstbx import visual

u = grstbx.utils
opj = os.path.join

grstbx.__version__

'2.0.1'

## Set Dask local cluster

In [2]:
#from dask.distributed import Client
#c = Client(n_workers=os.cpu_count()-2, threads_per_worker=1)
##cluster = dask.distributed.LocalCluster()
##client = dask.distributed.Client(cluster
#c.dashboard_link

## Set PROJ path if necessary

In [3]:
ppj.datadir.get_data_dir()
#ppj.datadir.set_data_dir('/work/scratch/harmelt/envs/grstbx/share/proj')

'/home/harmel/anaconda3/envs/grstbx/share/proj'

## Set the images you want to play with

In [4]:

tile='31TEJ'
idir='/home/harmel/Dropbox/satellite/S2'
select =grstbx.select_files()

# if you need to change the root path
select.root=idir
select.list_tile(tile=tile,product='L2Agrs',pattern='*20')
files = select.list_file_path(('2022-07','2022-08-31'))
files

pn.widgets.DataFrame(select.file_list, height=250,width=1000)

your path: /home/harmel/Dropbox/satellite/S2/L2Agrs/31TEJ/*/*/*/*20
wrong path, no data available; try again!


AttributeError: 'select_files' object has no attribute 'file_list'

In [ ]:
#l2driver = grstbx.driver.l2grs(files)
#prod,anc =l2driver.load_l2a_image(files[0])
#prod#.Rrs.rio.reproject(3857)

## Load and subset image series

In [ ]:
# central coordinate
lon, lat = 3.6,43.4
# size in meter of the rectangle of interest
width, height = 16000, 16000
aspect=width/height

ust = grstbx.utils.spatiotemp()
box = ust.wktbox(lon,lat, width=width, height=height, ellps='WGS84')
bbox = gpd.GeoSeries.from_wkt([box]).set_crs(epsg=4326)
# reproject lon lat in xy coordinates
#bbox = bbox.to_crs(epsg=32631)

In [ ]:
# generate datacube
dc = grstbx.l2grs(files)
dc.get_l2a_datacube(subset=bbox) #,reproject=True)

## Check data/metadata

## Mask datacube
Mask pixels from chosen flags and remove empty dates

In [ ]:
for param in ['Rrs','Rrs_g','BRDFg']:
    dc.datacube[param] = dc.datacube[param].where(dc.datacube.cirrus_band<0.005)


In [ ]:
dc.datacube.valid_pix_prop.plot()

In [ ]:
valid_time = dc.datacube.valid_pix_prop.where(dc.datacube.valid_pix_prop > 0.25,drop=True).time
dc.datacube = dc.datacube.sel(time=valid_time)

In [ ]:
coarsening = 1
cmap = plt.cm.Spectral_r
fig = dc.datacube.BRDFg[:,::coarsening, ::coarsening].plot.imshow(col='time', col_wrap=4,robust=True,cmap=cmap,aspect=aspect)
for ax in fig.axes.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig

## Reprojection into other coordinate reference systems

## **Fast checking of the RGB images**

In [ ]:
bands=[4,2,1]
#bands=[3,2,1]
coarsening = 1
brightness_factor = 5
gamma=2
fig = (dc.datacube.Rrs.isel(wl=bands)[:,:,::coarsening, ::coarsening]**(1/gamma)*brightness_factor).plot.imshow(col='time', col_wrap=4,robust=True,aspect=aspect)
for ax in fig.axes.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


In [ ]:
#Rrs_masked=Rrs_masked.drop_isel(time=[4,5,11,12,28])

In [ ]:
hv.extension('bokeh')
visual.image_viewer().Rrs_date(dc.datacube.Rrs)

## **Check spectral datacube** (i.e., Remote Sensing Reflectance, R<sub>rs</sub>, sr<sup>-1</sup>)

To quickly check your data visually, you can use the *visual* module of *grstbx*

In [ ]:
hv.extension('bokeh')
v=visual.view_spectral(dc.datacube.Rrs,reproject=True)

In [ ]:
v.visu()


In [ ]:
print(v.aoi_stream.data)


In [ ]:
geom_ = v.get_geom(v.aoi_stream,crs=dc.datacube.rio.crs)

Rrs_clipped = dc.datacube.Rrs.sel(wl=slice(400,1000)).rio.clip(geom_.geometry.values)

In [ ]:
bands=[4,2,1]
#bands=[3,2,1]
coarsening = 1
brightness_factor = 5
gamma=2
fig = (Rrs_clipped.isel(wl=bands)[:,:,::coarsening, ::coarsening]**(1/gamma)*brightness_factor).plot.imshow(col='time', col_wrap=4,robust=True,aspect=aspect)
for ax in fig.axes.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


In [ ]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in Rrs_clipped.data_vars}
filename='/data/satellite/S2/test_datacube_v2.nc'
Rrs_clipped.to_netcdf(filename, encoding=encoding)

In [ ]:
stacked = Rrs_clipped.dropna('time',thresh=0).stack(gridcell=["y", "x"]).dropna('gridcell',thresh=0).chunk(dict(gridcell=-1))


In [ ]:
stacked

In [ ]:
group_coord ='wl'
stat_coord='gridcell'
stats = xr.Dataset({'median':stacked.groupby(group_coord).median(stat_coord)})
stats['q25'] = stacked.groupby(group_coord).quantile(0.25,dim=stat_coord)
stats['q75'] = stacked.groupby(group_coord).quantile(0.75,dim=stat_coord)
stats['min'] = stacked.groupby(group_coord).min(stat_coord)
stats['max'] = stacked.groupby(group_coord).max(stat_coord)
stats['mean'] = stacked.groupby(group_coord).mean(stat_coord)
stats['std'] = stacked.groupby(group_coord).std(stat_coord)
stats['pix_num'] = stacked.count(stat_coord)

In [ ]:
%matplotlib inline
num_items = len(stats.time)
col_wrap=4
rows=int(np.ceil(num_items/col_wrap))

fig, axs = plt.subplots(nrows=rows,ncols=col_wrap, sharex=True,sharey=True, figsize=(20, rows*3.5))#,sharey=True
fig.subplots_adjust(hspace=0.1,wspace=0.1)
axs_ = axs.ravel()
[axi.set_axis_off() for axi in axs_]
for iax,(_,group) in enumerate(stats.groupby('time')): 
    date = group.time.dt.date.values
    
    axs_[iax].set_axis_on()
    axs_[iax].axhline(y=0,color='k',lw=1)
    axs_[iax].plot(group.wl,group['median'],c='k')
    axs_[iax].plot(group.wl,group['mean'],c='red',ls='--')
    axs_[iax].fill_between(group.wl, group['q25'], group['q75'],alpha=0.3,color='grey')
    axs_[iax].set_title(date)
plt.show()

In [ ]:
stats.to_netcdf('/DATA/git/insitu_app/shallowat/shallowat/data/s2_thau_Rrs_sand.nc')

In [ ]:
stats=stats.compute()

fig, axs = plt.subplots(1,3,figsize=(15,4),sharey=True)

for iax,(_,group) in enumerate(stats.groupby('time')): 
    date = group.time.dt.date.values
    axs[0].plot(group.wl,group['median'],label=date)
Rrs_mean=stats['median'].mean('time')
Rrs_q25=stats['median'].quantile(0.25,'time')
Rrs_q75=stats['median'].quantile(0.75,'time')

axs[1].fill_between(Rrs_mean.wl, Rrs_q25, Rrs_q75,alpha=0.3,color='grey')    
axs[1].plot(Rrs_mean.wl,Rrs_mean,c='k',lw=2)    
for ii in range(3):
    axs[ii].axhline(y=0,color='gray',lw=1)
    axs[ii].set_xlabel('$Wavelength\ (nm)$')

 
  
axs[0].set_ylabel('$R_{rs}\ (sr^{-1})$')

axs[0].legend(fontsize=8)

# L2B section

In [ ]:
tile='31TEJ'
idir='/data/satellite/S2/L2B'
select =grstbx.select_files()

# if you need to change the root path
select.root=idir
select.list_folder(pattern='*L2B*v20.nc')
files = select.list_file_path(('2022-07','2022-08-31'))
files

pn.widgets.DataFrame(select.file_list, height=250,width=1000)

In [ ]:

# load L2B images into datacube
dc = grstbx.l2grs(files)
dc.get_l2b_datacube(subset=bbox)

In [ ]:
dc.datacube.time

In [ ]:
cmap = plt.cm.Spectral_r
fig = dc.datacube.Chla_OC2nasa.plot.imshow(col='time', col_wrap=4,robust=True,cmap=cmap,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig

In [ ]:
v2b=visual.view_param(dc.datacube,reproject=True)

In [ ]:
v2b.visu()


In [ ]:
print(v2b.aoi_stream.data)
v2b.aoi_stream.element.geom()

In [ ]:
hv.Layout([ts().relabel(name) for name, ts in hv.element.tiles.tile_sources.items()]).opts(
    opts.Tiles(xaxis=None, yaxis=None, width=225, height=225)).cols(4)

In [ ]:
from holoviews import opts

opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='binary_r', width=800, height=700),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=900),
    opts.Overlay(show_legend=True))
# set the parameter for spectra extraction
hv.extension('bokeh')
pn.extension()

visual.image_viewer().Rrs_date(dc.datacube.Rrs)

# Check surface rugosity via sunglint BRDF

In [ ]:

raster = BRDFg#.isel(time=-1,drop=True)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'gray',colorbar=True)#.hist(bin_range=(0,0.02) ) 
widget = pn.widgets.RangeSlider(start=0, end=0.01,step=0.0005)

jscode = """
    color_mapper.low = cb_obj.value[0];
    color_mapper.high = cb_obj.value[1];
"""
link = widget.jslink(im, code={'value': jscode})
pn.Column(widget, im)

In [ ]:

shaded = []
for name, raster in BRDFg.groupby('time'):
    img = tf.shade(raster.squeeze(),cmap=cc.gray)
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 5
imgs

## Check blue over green ratio for Chl retrieval with OC2 from NASA
$log_{10}(chlor\_a) = a_0 + \sum\limits_{i=1}^4 a_i \left(log_{10}\left(\frac{R_{rs}(\lambda_{blue})}{R_{rs}(\lambda_{green})}\right)\right)^i$

In [ ]:
# NASA OC2 fro MODIS; bands 488, 547 nm
a = [0.2500,-2.4752,1.4061,-2.8233,0.5405]
# NASA OC2 for OCTS; bands 490, 565 nm
a = [0.2236,-1.8296,1.9094,-2.9481,-0.1718]

ratio = np.log10(Rrs_masked.isel(wl=1)/Rrs_masked.isel(wl=2))
logchl=0
for i in range(len(a)):
    logchl+=a[i]*ratio**i
chl = 10**(logchl)
chl.name='chl in mg.m-3 from OC2'

Set range of valid values

In [ ]:
chl = chl.where((chl >= 0) & (chl <= 80))
chl.persist()



In [ ]:
visual.image_viewer().param_date(chl,cmap='bgyw')

In [ ]:
raster = chl

shaded = []
for name, raster in chl.groupby('time'):
    img = tf.shade(raster.squeeze(),cmap=cc.bgyw, span=(0,10),how='log')
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 4
imgs

# CDOM retrieval based on Brezonik et al, 2015


In [ ]:
a = [1.872,-0.83]
acdom = np.exp(a[0] + a[1] * np.log(Rrs_masked.isel(wl=1)/Rrs_masked.isel(wl=5)))
acdom.name='CDOM absorption at 440 nm-1'
acdom= acdom.where((acdom >= 0) & (acdom <= 10))
acdom.persist()

In [ ]:
visual.image_viewer().param_date(acdom,cmap='bgyw')

# Total suspended particulate matter (SPM) from Nechad et al., 2010, 2016 formulation
spm in mg/L

In [ ]:
a = [610.94*np.pi, 0.2324/np.pi]
Rrs_ = Rrs_masked.isel(wl=3)
spm = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm.name='CDOM absorption at 440 nm-1'
spm= spm.where((spm >= 0) & (spm <= 150))
spm.persist()

In [ ]:
visual.image_viewer().param_date(spm,cmap='bgyw')

In [ ]:

shaded = []
for name, raster in dc.Rrs.Rrs.isel(time=-1).groupby('wl'):
    img = tf.shade(raster,cmap=cc.kbc)
    img.name = '{:.2f}'.format(name)+' nm'
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 5
imgs

In [ ]:
shaded = []
for name, raster in BRDFg.groupby('time'):
    img = tf.shade(raster,cmap=cc.gray, span=(0,0.025),how='log')
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 4
imgs

# Play with time series

In [ ]:
raster = spm

param = raster.name
third_dim = 'time'
time= raster.time.data
Ntime = len(time)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'RdBu_r',colorbar=True,clim=(0,100))#.hist(bin_range=(0,0.02) ) 

polys = hv.Polygons([])
box_stream = hv.streams.BoxEdit(source=polys)
dmap, dmap_std=[],[]

def roi_curves(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'time', param)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.nanmean).data
        print(mean)
        #std = selection.aggregate(third_dim, np.std).data
        time = mean[third_dim]

        curves[i]= hv.Curve((time,mean[param]),'time', param) 

    return hv.NdOverlay(curves)


# a bit dirty to have two similar function, but holoviews does not like mixing Curve and Spread for the same stream
def roi_spreads(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'time', param)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.nanmean).data
        std = selection.aggregate(third_dim, np.nanstd).data
        time = mean[third_dim]

        curves[i]=  hv.Spread((time,mean[param],std[param]),fill_alpha=0.3)

    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    


In [ ]:
# visualize and play
graphs = ((mean*std ).relabel(param))
layout = (im * polys +graphs    ).opts(
    opts.Curve(width=600, framewise=True), 
    opts.Polygons(fill_alpha=0.2, color='green',line_color='black'), 
    ).cols(2)
layout 